# Constraint Programming Task Scheduler

If you have a bunch of tasks to do in a week, and you have only so many hours in each day of the week, how do you figure out what to do and when?

This is a IPython notebook using constraint programming to answer that question.

- Tasks are defined with work effort in hours
- The schedule for the week is defined by assigning each day with available free hours. 


In [1387]:
from constraint import *
from itertools import *

# Configuration and Setup

In [1388]:
# Configure day naming constants
M,T,W,TH,F,Sat,Sun = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

In [1389]:
# Configure number of free hours each day
#
# Sample :
#   free_time[W] = 3  # Sets wednesday to 3 free hours

free_time = {}
free_time[M]  = 5  # Monday
free_time[T]  = 4  # Tuesday
free_time[W]  = 2  # Wednesday
free_time[TH] = 5  # Thursday
free_time[F]  = 1  # Friday
free_time[Sat]  = 0  # Saturday
free_time[Sun]  = 2  # Sunday

In [1390]:
# Configure tasks to complete this week
#
# Sample tasks
#   tasks['name'] = { 'effort': 1 }

tasks = {}
tasks['laundry'] = { 'effort': 2 }
tasks['pay bills'] = { 'effort': 2 }
tasks['wash the car'] = { 'effort': 3 }
tasks['mow the lawn'] = { 'effort': 3 }
tasks['write blog post'] = { 'effort': 1 }

# Precalculate intermediate values

In [1391]:
# Calculate helper variables
# Descriptions below
total_free_hours = 0
hour_to_day = {}

for day in [M,T,W,TH,F,Sat,Sun]:
    for hour in range(0,free_time[day]):
      hour_to_day[total_free_hours] = day
      total_free_hours += 1

hours_in_week = range(0,total_free_hours)

In [1392]:
# hour_to_day is a mapping of hour in week to a day
hour_to_day

{0: 'monday',
 1: 'monday',
 2: 'monday',
 3: 'monday',
 4: 'monday',
 5: 'tuesday',
 6: 'tuesday',
 7: 'tuesday',
 8: 'tuesday',
 9: 'wednesday',
 10: 'wednesday',
 11: 'thursday',
 12: 'thursday',
 13: 'thursday',
 14: 'thursday',
 15: 'thursday',
 16: 'friday',
 17: 'sunday',
 18: 'sunday'}

In [1393]:
# total_free_hours is a sum of all the free hours this week 
total_free_hours

19

In [1394]:
# hours_in_week is an incremental array of free hours this week
hours_in_week

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

# Defining the constraint solver

In [1395]:
problem = Problem()
# task.keys() , i.e. ['laundry', 'pay bills', ...] are the variables
# hours_in_week, i.e. [0,1,2,3,...] are the possible values for the variables
# 
# The constraint solver will try assigning different hours of the week
# to the variables, and reject all those that do not meet the constraints
problem.addVariables(tasks.keys(),hours_in_week)

## Add Constraint: Two tasks cannot begin at the same time

In [1396]:
# Add a constraint so that no task begins on an identical hour to another task
problem.addConstraint(AllDifferentConstraint())

## Add Constraint: A task needs enough time to complete before the week ends

In [1397]:
# Iterate over each task
# Add a constraint that validates the start time of a task allows enough time to complete it
for task_name in tasks.keys():
 problem.addConstraint(lambda start: start <= total_free_hours - tasks[task_name]['effort'], [task_name])

## Add Constraint: Two tasks can't be worked on at the same time

In [1398]:
# Iterate over each set of possible tasks
# Add a constraint that validates the start time + work effort is enough to complete the task
# before the next task should begin
for two_tasks in permutations(tasks.keys(),2):
    problem.addConstraint(lambda first_task, second_task: first_task + tasks[two_tasks[0]]['effort'] <= second_task if first_task < second_task else True, [two_tasks[0], two_tasks[1]])
    problem.addConstraint(lambda first_task, second_task: second_task + tasks[two_tasks[1]]['effort'] <= first_task if first_task > second_task else True, [two_tasks[0], two_tasks[1]])

# Solution

## Possible Number of Solutions

In [1399]:
len(problem.getSolutions())

15120

## Get Solution

In [1400]:
# Get a solution
solution = problem.getSolution()
solution

{'laundry': 16,
 'mow the lawn': 2,
 'pay bills': 13,
 'wash the car': 5,
 'write blog post': 10}

## Pretty Print Solution

In [1401]:
def print_schedule():
    prev_day = None
    for hour in hour_to_day.keys():
        # Print day
        if hour_to_day[hour] != prev_day :
            print "\n","== "+ hour_to_day[hour] + " =="
            prev_day = hour_to_day[hour]
        # Print task name
        current_task = None
        for task_name in solution:
            if (hour == solution[task_name]):
                if ( (current_task == None) or (solution[current_task] < solution[task_name]) ):
                    current_task = task_name
        if(current_task != None):
            print "  + ",current_task, tasks[current_task]
        else:
            print "  -"

# Print Schedule

The following is a possible schedule that will meet the constraints placed on the tasks.

Tasks are assumed to be started and flow through to the next date.
That is why you may see a task start on a certain day, but get completed on another.

In [1402]:
print_schedule()


== monday ==
  -
  -
  +  mow the lawn {'effort': 3}
  -
  -

== tuesday ==
  +  wash the car {'effort': 3}
  -
  -
  -

== wednesday ==
  -
  +  write blog post {'effort': 1}

== thursday ==
  -
  -
  +  pay bills {'effort': 2}
  -
  -

== friday ==
  +  laundry {'effort': 2}

== sunday ==
  -
  -
